In [23]:
import openai
from collections import deque

class Chatbot:
    def __init__(self, api_key, buffer_size=5):
        openai.api_key = api_key
        self.buffer = deque(maxlen=buffer_size)
        
    def get_response(self, prompt):
        # Using OpenAI API to get the response
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            prompt=prompt,
            max_tokens=150
        )
        message = response.choices[0].text.strip()
        self.buffer.append(message)
        return message
    
    def get_summary(self):
        # Summarize the conversation buffer
        summary_prompt = "Summarize this conversation: " + " ".join(self.buffer)
        summary = openai.chat.Completion.create(
            model="gpt-3.5-turbo",
            prompt=summary_prompt,
            max_tokens=150
        )
        return summary.choices[0].text.strip()


In [2]:
from pymongo import MongoClient
import uuid

class MongoDBHandler:
    def __init__(self, connection_string, database_name):
        self.client = MongoClient(connection_string)
        self.db = self.client[database_name]
        self.collection = self.db["conversations"]
        
    def store_conversation(self, session_id, conversation_summary):
        self.collection.insert_one({"session_id": session_id, "summary": conversation_summary})
        
    def get_conversation(self, session_id):
        return self.collection.find_one({"session_id": session_id})


In [3]:
from pymongo import MongoClient
import uuid

class MongoDBHandler:
    def __init__(self, connection_string, database_name):
        self.client = MongoClient(connection_string)
        self.db = self.client[database_name]
        self.collection = self.db["conversations"]
        
    def store_conversation(self, session_id, conversation_summary):
        self.collection.insert_one({"session_id": session_id, "summary": conversation_summary})
        
    def get_conversation(self, session_id):
        return self.collection.find_one({"session_id": session_id})


In [4]:
class ConversationManager:
    def __init__(self, chatbot, mongodb_handler):
        self.chatbot = chatbot
        self.mongodb_handler = mongodb_handler
        self.session_id = str(uuid.uuid4())
        
    def chat(self, prompt):
        response = self.chatbot.get_response(prompt)
        return response
    
    def end_conversation(self):
        summary = self.chatbot.get_summary()
        self.mongodb_handler.store_conversation(self.session_id, summary)
        return self.session_id


In [24]:
import os
# Initialize the components
api_key = os.getenv("OPENAI_API_KEY")
connection_string = "MONGODB_CONNECTION_STRING"
database_name = "chatbot_db"

chatbot = Chatbot(api_key)
mongodb_handler = MongoDBHandler(connection_string, database_name)
manager = ConversationManager(chatbot, mongodb_handler)

# Example conversation
print(manager.chat("Hello, how are you?"))
# print(manager.chat("Tell me a joke."))
# print(manager.chat("What is the weather like today?"))

# # End conversation and store in MongoDB
# session_id = manager.end_conversation()
# print(f"Session ID: {session_id}")


TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given